# Load LLM

In [ ]:
!pip install langchain langchain_community langgraph bitsandbytes

In [ ]:
import torch
import transformers

model_id = "codellama/CodeLlama-7b-Instruct-hf"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",
)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False,
    return_full_text=False,
    max_new_tokens=512
)

In [ ]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline)

# Phase 1: Security-Guided Prompt Construction

In [ ]:

import os
import json
import numpy as np
import pandas as pd

# ===============================
# CONFIGURATION
# ===============================
# Path to the precomputed knowledge base (KB) code embeddings (Scenario 2)
kb_embedding_path = '/kaggle/input/attackcode-sce2/merge_kb/embedding'

# Path to the KB source code files (used for retrieval)
kb_code_path = '/kaggle/input/attackcode-sce2/merge_kb'

# Programming languages to process (e.g., 'python', 'java', 'c', 'cpp')
languages = ['python']  # Adjust this list depending on which language(s) you're working with

# Directory to save the generated query embeddings
output_dir = "/kaggle/input/attackcode/query_embedding"

# Working directory for saving intermediate and output files (writeable)
results_dir = "/kaggle/working"

# Path to the CSV file that summarizes code samples with security warnings
summary_csv_file = "/kaggle/input/attackcode/all_generated_code_with_warning.csv"


os.makedirs(results_dir, exist_ok=True)
df_summary = pd.read_csv(summary_csv_file)
total_no_warning_count = 0

for lang in languages:
    print(f"\n🔍 Processing language: {lang.upper()}")
    try:
        # Load query embeddings
        query_embed_file = os.path.join(output_dir, f"{lang}_query_embeddings.npy")
        query_embeddings = np.load(query_embed_file)

        # Load KB embeddings
        kb_embed_file = os.path.join(kb_embedding_path, "cluster_full_codes_embed.npy")
        kb_embeddings = np.load(kb_embed_file)

        # Load KB raw code
        kb_code_file = os.path.join(kb_code_path, "cluster_full_codes_raw.json")
        with open(kb_code_file, 'r', encoding='utf-8') as f:
            kb_code_list = json.load(f)

        no_warning_data = []
        results = {}

        for idx, q_embed in enumerate(query_embeddings):
            similarities = np.dot(kb_embeddings, q_embed)
            top_indices = similarities.argsort()[-5:][::-1]

            query_results = []

            for rank, kb_idx in enumerate(top_indices, start=1):
                matched_code = kb_code_list[kb_idx]
                row_match = df_summary[df_summary['generated_code'] == matched_code]
                warning = row_match['warning'].values[0] if not row_match.empty else "ERROR"

                if warning == "ERROR":
                    no_warning_data.append(matched_code)

                query_results.append({
                    "kb_index": int(kb_idx),
                    "similarity": float(similarities[kb_idx]),
                    "code": matched_code,
                    "warning": warning
                })

            results[f"query_{idx + 1}"] = query_results

        no_warning_data = list(set(no_warning_data))
        total_no_warning_count += len(no_warning_data)

        output_file = os.path.join(results_dir, f"{lang}_query_results.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"\n✅ Results have been saved to: {output_file}")
        print(f"📊 Total number of queries: {len(results)}")


    except Exception as e:
        print(f"❌ Error processing {lang}: {e}")

print(f"\n📌 Total number of code snippets without warnings: {total_no_warning_count}")


In [ ]:
def trim_code(code, max_lines=100):
    lines = code.splitlines()
    return "\n".join(lines[:max_lines])

def prepare_langchain_inputs(query, search_results, lang="cpp"):
    # Collect unique warnings
    seen = set()
    warnings = []
    for result in search_results:
        warning = result.get("warning", "")
        if warning.strip() and warning != "⚠️ No warning found.":
            if warning not in seen:
                warnings.append(warning)
                seen.add(warning)

    # Format warning list
    warnings_text = "\n".join(f"{i+1}. {w}" for i, w in enumerate(warnings))

    examples = []
    for result in search_results:
        code = result.get("code", "")
        if code:
            trimmed = trim_code(code, max_lines=100)
            examples.append(trimmed)

    examples_text = "\n\n".join(examples)

    return {
        "lang": lang,
        "warnings": warning,
        "examples": examples_text,
        "question": query
    }


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import Runnable

template = """<s>[INST] <<SYS>>
You are a security-focused coding assistant specialized in {lang}. Only output valid {lang} code that resolves all of the warnings listed below.
Do not include any explanation — just the code.
<</SYS>>
Here is the context:
```
{examples}
```
Warnings to avoid:
{warnings}

Now, answer the following request based on the above context and warnings:

{question}
[/INST]"""


prompt = PromptTemplate.from_template(template)
chain: Runnable = prompt | llm | StrOutputParser()


In [ ]:
def generate_code(query, lang, query_index):

    result_file = os.path.join(results_dir, f"{lang}_query_results.json")
    with open(result_file, "r", encoding="utf-8") as f:
        all_results = json.load(f)

    query_key = f"query_{query_index + 1}"
    if query_key not in all_results:
        raise ValueError(f"Không tìm thấy kết quả cho {query_key} trong {result_file}")

    top_k_results = all_results[query_key]

    inputs = prepare_langchain_inputs(query, top_k_results, lang=lang)

    return chain.invoke(inputs)

# =============================
# 4. Main loop
# =============================
query_folder= "/kaggle/input/attackcode/query"
def answer_all(lang):
    query_file = f"{query_folder}/{lang}_query.json"
    output_file = f"/codellama/answer/{lang}_output_sce1.json"
    flat_code_file = os.path.join(results_dir, f"{lang}_flat_code_output.json")

    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Load queries
    with open(query_file, "r", encoding="utf-8") as f:
        queries = json.load(f)

    all_answers = {}
    code_list = []
    for idx, query in enumerate(tqdm(queries)):
        try:
            answer = generate_code(query, lang, idx)
            all_answers[query] = answer
            code_list.append(answer.strip())
        except Exception as e:
            print(f"⚠️ Error in query {idx}: {query[:50]}... -> Detail: {e}")

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_answers, f, indent=2, ensure_ascii=False)

    with open(flat_code_file, "w", encoding="utf-8") as f:
        json.dump(code_list, f, indent=2, ensure_ascii=False)



# Phase 2: Code Generation with Self-Reflection and Refinement

In [ ]:
from tqdm import tqdm


for lang in languages:
  answer_all(lang)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import Runnable

template = """<s>[INST] <<SYS>>
You are a software security expert and experienced programmer.
<</SYS>>
Your task is to:
1. Perform a detailed scan to summarize any potential vulnerabilities or security flaws found in the following code.
2. Suggest a concise way to fix the issues, but do not rewrite the entire code.

Format your response exactly like this:
Vulnerability Description: [summary of the issue]
Fix Suggestion: [short suggestion to fix]

Code to analyze vulnerabilities:
{code}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
chain: Runnable = prompt | llm | StrOutputParser()

In [ ]:
# Build the file path dynamically based on the selected language
file_path = f"/kaggle/working/{languages[0]}_flat_code_output.json"

# Load the list of code snippets from the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    code_list = json.load(f)

In [ ]:
from tqdm import tqdm

results = []

for idx, code in enumerate(tqdm(code_list)):
    try:
        result = chain.invoke({"code": code})
        results.append({
            "index": idx,
            "vulnerability_analysis": result.strip(),
            "original_code": code
        })
    except Exception as e:
        print(f"⚠️ Error in code {idx}: {e}")
        results.append({
            "index": idx,
            "vulnerability_analysis": "Error",
            "original_code": code
        })


# Build the output path dynamically based on the selected language
output_path = f"/kaggle/working/{languages[0]}_flat_analysis_output.json"

# Save the results to a JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import Runnable

template_fix = """<s>[INST] <<SYS>>
You are a security code fixer. Output only valid {lang} code — no comments, no explanation.
<</SYS>>
Fix the code below using the vulnerability report. Improve security and code quality. Return only the corrected {lang} code.

Original Code:
{generated_code}

Vulnerability Report:
{vulnerability_report}
[/INST]
"""

fix_prompt = PromptTemplate.from_template(template_fix)
fix_chain: Runnable = fix_prompt | llm | StrOutputParser()


In [ ]:
import json
from tqdm import tqdm

language = languages[0].
# Build input and output paths dynamically based on language
input_path = f"/kaggle/working/{languages[0]}_flat_analysis_output.json"
output_path = f"/kaggle/working/{languages[0]}_fixed_code_output.json"

with open(input_path, "r", encoding="utf-8") as f:
    analysis_results = json.load(f)

fixed_results = []
for row in tqdm(analysis_results):
    try:
        inputs = {
            "generated_code": row["original_code"],
            "vulnerability_report": row["vulnerability_analysis"],
            "lang": language
        }
        fixed_code = fix_chain.invoke(inputs)
        fixed_results.append({
            "index": row["index"],
            "fixed_code": fixed_code.strip(),
            "original_code": row["original_code"],
            "vulnerability_report": row["vulnerability_analysis"]
        })
    except Exception as e:
        print(f"⚠️Error in index {row.get('index')}: {e}")
        fixed_results.append({
            "index": row.get("index"),
            "fixed_code": "Error",
            "original_code": row["original_code"],
            "vulnerability_report": row["vulnerability_analysis"]
        })

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(fixed_results, f, indent=2, ensure_ascii=False)

print(f"Results are save to: {output_path}")
